# Deep Learning Tutorial 1: Build an Image Recognizer on your dataset in less 10 minutes

This tutorial is a concise step-by-step guide for building and training an Image Recongizer on **any** image dataset of your choice.<br>

In this tutorial, you will learn how to:

- Scrape images from Google Images and create your own dataset
- Build and train an image recognizer on your dataset
- Visualize and adequately interpret classification results
- Test model with new images

To simply run this notebook, you can download it [here](https://drive.google.com/open?id=1MNpxqMGvg2IanZfmLmtpUFqjUUGb3Wuv) and upload it to your Google [Colab](https://colab.research.google.com/notebooks/welcome.ipynb#recent=true). <br>Once uploaded to Colab, **make sure to change the following to enable GPU backend,** <br>
> Runtime -> Change runtime type -> Hardware Accelerator -> GPU

## 1. Image Dataset Download and Setup

**Option 1:** Working with a ready dataset of your own

If you would like to use your own image dataset for this tutorial, rearrange it in a way that images of same class are under same folder. Then, name the folders with the corresponding class labels.

**Option 2:** Scrapping images from Google Images and making up a dataset

If you do not have a dataset in-hand, you can scrape images from [Google Images](https://www.google.com/imghp?hl=en) and make up a dataset of your choice. In order to do so, simply install [Fatkun Batch Download Image](https://chrome.google.com/webstore/detail/fatkun-batch-download-ima/nnjjahlikiabnchcpehcpkdeckfgnohf) extension on your google chrome browser and download any set of google images by clicking the extension tab. Place all images of same class under same folder and name it accordingly with the class label.  

**Option 3:** Working with my Monkeys Faces dataset :)

You can easily download the dataset folder by uncommenting the command on the next cell. Then, upload the downloaded folder to your Colab.

In [17]:
%%capture
!wget "https://www.dropbox.com/s/35ithckx6vqryob/Monkeys_Faces_Dataset.tar?dl=0" -O Monkeys_Faces_Dataset.tar  
!tar -xvzf Monkeys_Faces_Dataset.tar

In this tutorial, we will be classifying **Monkeys Faces**, so we created 6 folders corresponding to the 6 different types of monkeys, as shown below.<br><br>

<img width="386" alt="Tutorial1_img1" src="https://user-images.githubusercontent.com/8379428/56394394-1c827a80-61ec-11e9-8cf2-42c0b87fffb6.png">

If you are running this on Colab, have the dataset saved in your google drive. If you are running this locally or at some VM, save the dataset on the path of your choice or preferebaly in the same path of this notebook.

## 2. Image Recognition

**For a further explanation of the code and functions used within this notebook, check out my other post: [Concise Notes on Image Recognition, Fastai 2019 v3 Lesson 1](https://github.com/SalChem/Fastai-iNotes-iTutorials/blob/master/Image_Recognition_Basics.ipynb)**

### Initialization

In [18]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
from fastai.vision import *
from fastai.metrics import error_rate

In [20]:
bs = 64  #batch size: if your GPU is running out of memory, set a smaller batch size, i.e 16
sz = 224 #image size
PATH = './Monkeys Faces'

PATH is the path containing all the class folders. You may keep './Monkeys Faces' if you are using my Monkeys Faces dataset.

Let's retrieve your image classes,

In [21]:
os.listdir(PATH)

['.DS_Store',
 'Macaque',
 '._Orungatun',
 'Orungatun',
 '._Macaque',
 'Chimpanzee',
 'Gorilla',
 '._Chimpanzee',
 '._.DS_Store',
 '._Gorilla',
 '._Gibbon',
 'Mandrill',
 '._Mandrill',
 'Gibbon']

In [23]:
classes = []
for d in os.listdir(PATH):
    print (d)
    if os.path.isdir(d) and not d.startswith('.'):
        classes.append(d) 
        #print ("here: ", d)

print ("There are ", len(classes), "classes:", classes)            

.DS_Store
Macaque
._Orungatun
Orungatun
._Macaque
Chimpanzee
Gorilla
._Chimpanzee
._.DS_Store
._Gorilla
._Gibbon
Mandrill
._Mandrill
Gibbon
There are  0 classes: []


In [45]:
pwd

'/home/jupyter/Cloud-Fastai-iNotes-iTutorials/Fastai-iNotes-iTutorials'

Let's verify there are not any corrupt images that cannot be read. If found any, they will simply be deleted.

In [5]:
for c in classes:
    verify_images(c, delete=True);

### Creating and training the classifier

Let's create the training and validation sets,

In [6]:
data  = ImageDataBunch.from_folder(PATH, ds_tfms=get_transforms(), size=sz, bs=bs, valid_pct=0.2).normalize(imagenet_stats)

/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py:438: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


IndexError: tuple index out of range

In [ ]:
print ("There are", len(data.train_ds), "training images and", len(data.valid_ds), "validation images." )

Let's visualize some images from different classes,

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

Let's build our Deep Convolutional Neural Network (CNN),

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

Let's pick the appropriate learning rate parameter,

In [ ]:
learn.lr_find();
learn.recorder.plot()

Use the above plot to pick adequate learning rates for your model, we need two learning rates:

- The first learning rate is just before the loss starts to increase, preferrably 10x smaller than the rate at which the loss starts to increase. For instance, 1e-02 for our Monkeys Face Recognition dataset. 

- The second learning rate is 10x smaller than the first learning rate, so 1e-03 in our example.

The plot will be different for your dataset, so make sure to pick these two learning rates accordingly.

<br>Replace your chosen learning rates in the **slice** fucntion below and let's train the model

In [ ]:
learn.fit_one_cycle(6, max_lr=slice(1e-3,1e-2))

Great, we achieved an accuracy of 94% on our Monkeys Faces dataset with only a few lines of code and without much tuning of the parameters.

We are DONE but let's further interpret the results.

## Results Interpretation and Visualization

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

We can start by visualizing a confusion matrix. The diagonal elements represent the number of images for which the predicted label is equal to the true label, while off-diagonal elements are those that are mislabeled by the classifier.

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

We can also plot images with top losses. A high loss implies high confidence about the wrong answer. 

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

Images are shown along with top losses:<br> 
prediction label / actual label / loss / probability of actual image class.

#### Congratulations! You have successfully built an Image Recognizer on your own dataset. 

## Testing the model on a new image

Let's now feed the model a new image it never seen before and see how well it classifies it. Place the image within your desired path.

In [ ]:
path = '../Test Images/' #The path of your test image

img = open_image(get_image_files(path)[0])

pred_class,pred_idx,outputs = learn.predict(img)
pred_class

img.show()

Great job completing this tutorial. If you liked and want similar tutorials, please give it few Claps or comment below.

Acknowledgment: This tutorial uses [fastai library](https://docs.fast.ai/index.html), so we thank Jeremy Howard for creating all the beautiful [fastai](https://www.fast.ai/) content.